In [9]:
import subprocess
import json
import os
import re
import pandas as pd
from typing import Dict, List, Optional, Tuple

# hwp5txt 절대 경로
HWP5TXT_PATH = "/home/spai0323/myenv/bin/hwp5txt"

def extract_hwp_text_advanced(hwp_path):
    """hwp5txt로 HWP 텍스트 추출 - 다양한 옵션 시도"""
    try:
        # 기본 추출 시도
        result = subprocess.run(
            [HWP5TXT_PATH, hwp_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"[실패] {hwp_path} : {e.stderr}")
        return None

def extract_hwp_structured(hwp_path):
    """hwp5 라이브러리로 구조화된 데이터 추출 시도"""
    try:
        # hwp5 라이브러리가 있다면 구조화된 추출 시도
        result = subprocess.run(
            ["hwp5html", hwp_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        if result.returncode == 0:
            return result.stdout, "html"
    except:
        pass
    
    # 일반 텍스트 추출
    text = extract_hwp_text_advanced(hwp_path)
    return text, "text" if text else None

def detect_table_patterns(text: str) -> List[Tuple[int, int, str]]:
    """다양한 표 패턴 감지"""
    table_patterns = []
    lines = text.split('\n')
    
    # 패턴 1: 연속된 탭으로 구분된 행들
    tab_table_start = None
    for i, line in enumerate(lines):
        if '\t' in line and len(line.split('\t')) >= 2:
            if tab_table_start is None:
                tab_table_start = i
        else:
            if tab_table_start is not None:
                if i - tab_table_start >= 2:  # 최소 2행 이상
                    table_patterns.append((tab_table_start, i, 'tab_separated'))
                tab_table_start = None
    
    # 마지막 표가 파일 끝까지 이어지는 경우
    if tab_table_start is not None and len(lines) - tab_table_start >= 2:
        table_patterns.append((tab_table_start, len(lines), 'tab_separated'))
    
    # 패턴 2: 공백으로 정렬된 표
    aligned_table_start = None
    for i, line in enumerate(lines):
        # 3개 이상의 연속 공백으로 구분된 컬럼이 있는 행
        if re.search(r'\S+\s{3,}\S+\s{3,}\S+', line):
            if aligned_table_start is None:
                aligned_table_start = i
        else:
            if aligned_table_start is not None:
                if i - aligned_table_start >= 2:
                    table_patterns.append((aligned_table_start, i, 'space_aligned'))
                aligned_table_start = None
    
    # 패턴 3: 특정 키워드로 시작하는 표
    table_keywords = ['번호', '구분', '항목', '내용', '날짜', '금액', '수량']
    for i, line in enumerate(lines):
        line_clean = line.strip()
        if any(keyword in line_clean for keyword in table_keywords):
            # 다음 몇 줄이 표 데이터인지 확인
            table_end = i + 1
            while table_end < len(lines) and (lines[table_end].strip() and 
                  ('\t' in lines[table_end] or re.search(r'\S+\s{2,}\S+', lines[table_end]))):
                table_end += 1
            if table_end - i >= 2:
                table_patterns.append((i, table_end, 'keyword_table'))
    
    return table_patterns

def extract_table_data(lines: List[str], start: int, end: int, pattern_type: str) -> List[List[str]]:
    """표 데이터 추출 및 정리"""
    table_data = []
    
    for i in range(start, end):
        line = lines[i].strip()
        if not line:
            continue
            
        if pattern_type == 'tab_separated':
            row = [cell.strip() for cell in line.split('\t')]
        elif pattern_type == 'space_aligned':
            # 3개 이상의 연속 공백으로 분할
            row = [cell.strip() for cell in re.split(r'\s{3,}', line) if cell.strip()]
        elif pattern_type == 'keyword_table':
            # 탭이 있으면 탭으로, 없으면 공백으로 분할
            if '\t' in line:
                row = [cell.strip() for cell in line.split('\t')]
            else:
                row = [cell.strip() for cell in re.split(r'\s{2,}', line) if cell.strip()]
        else:
            row = [line]
        
        if row and any(cell for cell in row):  # 빈 행 제외
            table_data.append(row)
    
    return table_data

def normalize_table(table_data: List[List[str]]) -> List[List[str]]:
    """표 데이터 정규화 (컬럼 수 맞추기)"""
    if not table_data:
        return table_data
    
    max_cols = max(len(row) for row in table_data)
    
    normalized = []
    for row in table_data:
        # 부족한 컬럼은 빈 문자열로 채움
        normalized_row = row + [''] * (max_cols - len(row))
        normalized.append(normalized_row)
    
    return normalized

def parse_text_and_tables_improved(text: str) -> Dict:
    """개선된 본문과 표 파싱"""
    lines = text.split('\n')
    parsed = {"paragraphs": [], "tables": []}
    
    # 표 패턴 감지
    table_patterns = detect_table_patterns(text)
    
    # 표가 아닌 텍스트 라인들을 추적
    table_lines = set()
    for start, end, _ in table_patterns:
        table_lines.update(range(start, end))
    
    # 표 데이터 추출
    for i, (start, end, pattern_type) in enumerate(table_patterns):
        table_data = extract_table_data(lines, start, end, pattern_type)
        if table_data:
            normalized_table = normalize_table(table_data)
            table_info = {
                "table_id": i + 1,
                "pattern_type": pattern_type,
                "start_line": start,
                "end_line": end,
                "headers": normalized_table[0] if normalized_table else [],
                "data": normalized_table[1:] if len(normalized_table) > 1 else [],
                "raw_data": normalized_table
            }
            parsed["tables"].append(table_info)
    
    # 표가 아닌 텍스트를 단락으로 수집
    current_paragraph = []
    for i, line in enumerate(lines):
        if i not in table_lines:
            line = line.strip()
            if line:
                current_paragraph.append(line)
            else:
                if current_paragraph:
                    parsed["paragraphs"].append(' '.join(current_paragraph))
                    current_paragraph = []
    
    # 마지막 단락 처리
    if current_paragraph:
        parsed["paragraphs"].append(' '.join(current_paragraph))
    
    return parsed

def export_tables_to_csv(tables: List[Dict], output_folder: str, base_filename: str):
    """표를 개별 CSV 파일로 저장"""
    csv_folder = os.path.join(output_folder, "csv_tables")
    os.makedirs(csv_folder, exist_ok=True)
    
    for table in tables:
        if table["raw_data"]:
            table_filename = f"{base_filename}_table_{table['table_id']}.csv"
            csv_path = os.path.join(csv_folder, table_filename)
            
            # pandas로 CSV 저장
            df = pd.DataFrame(table["raw_data"])
            df.to_csv(csv_path, index=False, header=False, encoding='utf-8-sig')
            print(f"  표 {table['table_id']} -> {table_filename}")

def hwp_folder_to_json_improved(raw_folder: str, processed_folder: str):
    """개선된 HWP to JSON 변환"""
    json_output_folder = os.path.join(processed_folder, "json_output")
    os.makedirs(json_output_folder, exist_ok=True)
    
    hwp_files = [f for f in os.listdir(raw_folder) if f.endswith(".hwp")]
    
    summary = {
        "total_files": len(hwp_files),
        "processed_files": 0,
        "failed_files": 0,
        "total_tables_found": 0,
        "files_with_tables": 0
    }
    
    for hwp_file in hwp_files:
        hwp_path = os.path.join(raw_folder, hwp_file)
        print(f"처리 중: {hwp_file}")
        
        # 텍스트 추출
        result = extract_hwp_structured(hwp_path)
        if result[0] is None:
            summary["failed_files"] += 1
            continue
        
        text, format_type = result
        
        # 개선된 파싱
        parsed = parse_text_and_tables_improved(text)
        
        # 데이터 구성
        data = {
            "filename": hwp_file,
            "format_type": format_type,
            "extraction_summary": {
                "total_paragraphs": len(parsed["paragraphs"]),
                "total_tables": len(parsed["tables"]),
                "table_types": [t["pattern_type"] for t in parsed["tables"]]
            },
            "content": parsed
        }
        
        # JSON 저장
        json_filename = os.path.splitext(hwp_file)[0] + ".json"
        json_path = os.path.join(json_output_folder, json_filename)
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        # 표가 있는 경우 CSV로도 저장
        if parsed["tables"]:
            base_filename = os.path.splitext(hwp_file)[0]
            export_tables_to_csv(parsed["tables"], processed_folder, base_filename)
            summary["files_with_tables"] += 1
            summary["total_tables_found"] += len(parsed["tables"])
        
        summary["processed_files"] += 1
        print(f"  완료: {len(parsed['paragraphs'])}개 단락, {len(parsed['tables'])}개 표")
    
    # 요약 정보 저장
    summary_path = os.path.join(processed_folder, "processing_summary.json")
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    
    print(f"\n=== 처리 완료 ===")
    print(f"총 파일: {summary['total_files']}")
    print(f"성공: {summary['processed_files']}")
    print(f"실패: {summary['failed_files']}")
    print(f"표가 포함된 파일: {summary['files_with_tables']}")
    print(f"총 표 개수: {summary['total_tables_found']}")

# 사용 예시
if __name__ == "__main__":
    raw_folder = "../data/raw/files""
    processed_folder = "../data/processed"

    # pandas 설치 확인
    try:
        import pandas as pd
        print("pandas가 설치되어 있습니다.")
    except ImportError:
        print("pandas를 설치해주세요: pip install pandas")
        exit(1)
    
    hwp_folder_to_json_improved(raw_folder, processed_folder)
    print("모든 HWP 파일이 처리되었습니다.")
    print("- JSON: data/processed/json_output/")
    print("- CSV 표: data/processed/csv_tables/")
    print("- 요약: data/processed/processing_summary.json")

pandas가 설치되어 있습니다.
처리 중: 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp
  표 1 -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구_table_1.csv
  표 2 -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구_table_2.csv
  표 3 -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구_table_3.csv
  표 4 -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구_table_4.csv
  표 5 -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구_table_5.csv
  표 6 -> 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구_table_6.csv
  완료: 307개 단락, 6개 표
처리 중: 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .hwp
  표 1 -> 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 _table_1.csv
  표 2 -> 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 _table_2.csv
  완료: 302개 단락, 2개 표
처리 중: 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.hwp
  표 1 -> 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입_table_1.csv
  표 2 -> 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입_table_2.csv
  완료: 373개 단락, 2개 표
처리 중: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.hwp
  표 1 -> 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축_table_1.csv
  표 2 -> 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시

KeyboardInterrupt: 

In [14]:
import shutil

# 삭제할 폴더 경로를 지정하세요.
folder_path = '/home/spai0323/projectmission2/data/processed/json_output'

try:
    shutil.rmtree(folder_path)
    print(f"'{folder_path}' 폴더와 모든 내용이 성공적으로 삭제되었습니다.")
except OSError as e:
    print(f"오류: {e.strerror} - {e.filename}")

'/home/spai0323/projectmission2/data/processed/json_output' 폴더와 모든 내용이 성공적으로 삭제되었습니다.
